In [1]:
import pandas as pd
import requests
import time

# === CONFIG ===
API_KEY = "PJcSVU8gR4wG9S9HVANWyssG62n2M6Sy"
TICKERS_URL = "https://raw.githubusercontent.com/datasets/s-and-p-500-companies/master/data/constituents.csv"
LIMIT = 1  # number of past reports (1 = latest)
SLEEP_TIME = 1  # avoid rate limit

# === LOAD TICKERS ===
print("Loading S&P 500 tickers...")
tickers_df = pd.read_csv(TICKERS_URL)
tickers = tickers_df['Symbol'].tolist()

# === FUNCTION TO FETCH DATA ===
def get_fmp_data(endpoint, symbol):
    url = f"https://financialmodelingprep.com/api/v3/{endpoint}/{symbol}?limit={LIMIT}&apikey={API_KEY}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if data:
            return data[0]
    return {}

# === MAIN LOOP ===
all_data = []

print("Fetching data for each ticker...")
for i, symbol in enumerate(tickers[230:310]):
    print(f"[{i+1}/{len(tickers)}] {symbol}")

    income = get_fmp_data("income-statement", symbol)
    balance = get_fmp_data("balance-sheet-statement", symbol)
    cashflow = get_fmp_data("cash-flow-statement", symbol)

    if income or balance or cashflow:
        merged = {
            "symbol": symbol,
            "reportDate": income.get("date", balance.get("date", cashflow.get("date", None))),
            **income,
            **{f"balance_{k}": v for k, v in balance.items()},
            **{f"cashflow_{k}": v for k, v in cashflow.items()}
        }
        all_data.append(merged)

    time.sleep(SLEEP_TIME)

# === SAVE RESULTS ===
df_230_310 = pd.DataFrame(all_data)
df_230_310.to_csv("230_310_sp500_fundamentals.csv", index=False)
print("✅ Data saved to '230_310_sp500_fundamentals.csv'")


Loading S&P 500 tickers...
Fetching data for each ticker...
[1/503] HLT
[2/503] HOLX
[3/503] HD
[4/503] HON
[5/503] HRL
[6/503] HST
[7/503] HWM
[8/503] HPQ
[9/503] HUBB
[10/503] HUM
[11/503] HBAN
[12/503] HII
[13/503] IBM
[14/503] IEX
[15/503] IDXX
[16/503] ITW
[17/503] INCY
[18/503] IR
[19/503] PODD
[20/503] INTC
[21/503] ICE
[22/503] IFF
[23/503] IP
[24/503] IPG
[25/503] INTU
[26/503] ISRG
[27/503] IVZ
[28/503] INVH
[29/503] IQV
[30/503] IRM
[31/503] JBHT
[32/503] JBL
[33/503] JKHY
[34/503] J
[35/503] JNJ
[36/503] JCI
[37/503] JPM
[38/503] JNPR
[39/503] K
[40/503] KVUE
[41/503] KDP
[42/503] KEY
[43/503] KEYS
[44/503] KMB
[45/503] KIM
[46/503] KMI
[47/503] KKR
[48/503] KLAC
[49/503] KHC
[50/503] KR
[51/503] LHX
[52/503] LH
[53/503] LRCX
[54/503] LW
[55/503] LVS
[56/503] LDOS
[57/503] LEN
[58/503] LII
[59/503] LLY
[60/503] LIN
[61/503] LYV
[62/503] LKQ
[63/503] LMT
[64/503] L
[65/503] LOW
[66/503] LULU
[67/503] LYB
[68/503] MTB
[69/503] MPC
[70/503] MKTX
[71/503] MAR
[72/503] MMC
[73/5

In [2]:
df_230_310.shape

(80, 133)

In [3]:
df_230_310.head()

,symbol,reportDate,date,reportedCurrency,cik,fillingDate,acceptedDate,calendarYear,period,revenue,...,cashflow_netCashUsedProvidedByFinancingActivities,cashflow_effectOfForexChangesOnCash,cashflow_netChangeInCash,cashflow_cashAtEndOfPeriod,cashflow_cashAtBeginningOfPeriod,cashflow_operatingCashFlow,cashflow_capitalExpenditure,cashflow_freeCashFlow,cashflow_link,cashflow_finalLink
0,HLT,2024-12-31,2024-12-31,USD,0001585689,2025-02-06,2025-02-06 10:05:57,2024,FY,11174000000,...,-1045000000,-21000000,501000000,1376000000,875000000,2013000000,-1.980000e+08,1815000000,https://www.sec.gov/Archives/edgar/data/158568...,https://www.sec.gov/Archives/edgar/data/158568...
1,HOLX,2024-09-28,2024-09-28,USD,0000859737,2024-11-27,2024-11-27 07:24:17,2024,FY,4030300000,...,-1108600000,8900000,-595500000,2160200000,2755700000,1285200000,-1.402000e+08,1145000000,https://www.sec.gov/Archives/edgar/data/859737...,https://www.sec.gov/Archives/edgar/data/859737...
2,HD,2025-02-02,2025-02-02,USD,0000354950,2025-03-21,2025-03-20 18:36:42,2024,FY,159514000000,...,-694000000,-186000000,-2101000000,1659000000,3760000000,19810000000,-3.485000e+09,16325000000,https://www.sec.gov/Archives/edgar/data/354950...,https://www.sec.gov/Archives/edgar/data/354950...
3,HON,2024-12-31,2024-12-31,USD,0000773840,2025-02-14,2025-02-14 14:50:25,2024,FY,38498000000,...,6839000000,-137000000,2642000000,10567000000,7925000000,6097000000,-1.164000e+09,4933000000,https://www.sec.gov/Archives/edgar/data/773840...,https://www.sec.gov/Archives/edgar/data/773840...
4,HRL,2024-10-27,2024-10-27,USD,0000048465,2024-12-05,2024-12-05 14:40:07,2024,FY,11920797000,...,-1030096000,5614000,5349000,741881000,736532000,1266738000,-2.564410e+08,1010297000,https://www.sec.gov/Archives/edgar/data/48465/...,https://www.sec.gov/Archives/edgar/data/48465/...


Ho runnato solo i primi 310, poi vanno mergiati tutti insieme

# -------- PROVA CON SOLO TICKER APPLE --------

In [13]:
import pandas as pd
import requests

# === CONFIG ===
API_KEY = "PJcSVU8gR4wG9S9HVANWyssG62n2M6Sy"
symbol = "AAPL"

# === FUNCTION TO FETCH DATA ===
def get_fmp_data(endpoint, symbol):
    url = f"https://financialmodelingprep.com/api/v3/{endpoint}/{symbol}?limit=1&apikey={API_KEY}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if data:
            return data[0]  # solo l'ultimo anno
    return {}

# === FETCH DATA ===
print(f"Fetching annual fundamentals for {symbol}")

income = get_fmp_data("income-statement", symbol)
balance = get_fmp_data("balance-sheet-statement", symbol)
cashflow = get_fmp_data("cash-flow-statement", symbol)

# === MERGE & CREATE DATAFRAME ===
merged = {
    "symbol": symbol,
    "reportDate": income.get("date", balance.get("date", cashflow.get("date", None))),
    **income,
    **{f"balance_{k}": v for k, v in balance.items()},
    **{f"cashflow_{k}": v for k, v in cashflow.items()}
}

df_apple = pd.DataFrame([merged])  # salva in DataFrame con una riga
print(df_apple.head())  # anteprima



Fetching annual fundamentals for AAPL
  symbol  reportDate        date reportedCurrency         cik fillingDate  \
0   AAPL  2024-09-28  2024-09-28              USD  0000320193  2024-11-01   

          acceptedDate calendarYear period       revenue  ...  \
0  2024-11-01 06:01:36         2024     FY  391035000000  ...   

   cashflow_netCashUsedProvidedByFinancingActivities  \
0                                      -121983000000   

   cashflow_effectOfForexChangesOnCash  cashflow_netChangeInCash  \
0                                    0                -794000000   

   cashflow_cashAtEndOfPeriod  cashflow_cashAtBeginningOfPeriod  \
0                 29943000000                       30737000000   

   cashflow_operatingCashFlow  cashflow_capitalExpenditure  \
0                118254000000                  -9447000000   

   cashflow_freeCashFlow                                      cashflow_link  \
0           108807000000  https://www.sec.gov/Archives/edgar/data/320193...   

       

In [14]:
df_apple.columns

Index(['symbol', 'reportDate', 'date', 'reportedCurrency', 'cik',
       'fillingDate', 'acceptedDate', 'calendarYear', 'period', 'revenue',
       ...
       'cashflow_netCashUsedProvidedByFinancingActivities',
       'cashflow_effectOfForexChangesOnCash', 'cashflow_netChangeInCash',
       'cashflow_cashAtEndOfPeriod', 'cashflow_cashAtBeginningOfPeriod',
       'cashflow_operatingCashFlow', 'cashflow_capitalExpenditure',
       'cashflow_freeCashFlow', 'cashflow_link', 'cashflow_finalLink'],
      dtype='object', length=133)

## ---------- VARIABILI IMPORTANTI ED ESEMPI ------------

### VARIABILI IMPORTANTI PER INCOME

Variabile | Descrizione | Frequenza reale di variazione

revenue | Ricavi totali della società | Trimestrale (earnings)

grossProfit | Ricavi - costo del venduto | Trimestrale

operatingIncome | Risultato operativo prima delle tasse e interessi | Trimestrale

netIncome | Utile netto | Trimestrale

eps (Earnings per Share) | Utile per azione | Trimestrale

interestExpense | Interessi passivi | Trimestrale

In [ ]:
# Esempio di output per income

[
    {
        "date": "2024-09-28",
        "symbol": "AAPL",
        "reportedCurrency": "USD",
        "cik": "0000320193",
        "fillingDate": "2024-11-01",
        "acceptedDate": "2024-11-01 06:01:36",
        "calendarYear": "2024",
        "period": "FY",
        "revenue": 391035000000,
        "costOfRevenue": 210352000000,
        "grossProfit": 180683000000,
        "grossProfitRatio": 0.4620634982,
        "researchAndDevelopmentExpenses": 31370000000,
        "generalAndAdministrativeExpenses": 0,
        "sellingAndMarketingExpenses": 0,
        "sellingGeneralAndAdministrativeExpenses": 26097000000,
        "otherExpenses": 0,
        "operatingExpenses": 57467000000,
        "costAndExpenses": 267819000000,
        "interestIncome": 0,
        "interestExpense": 0,
        "depreciationAndAmortization": 11445000000,
        "ebitda": 134661000000,
        "ebitdaratio": 0.3443707085,
        "operatingIncome": 123216000000,
        "operatingIncomeRatio": 0.3151022287,
        "totalOtherIncomeExpensesNet": 269000000,
        "incomeBeforeTax": 123485000000,
        "incomeBeforeTaxRatio": 0.3157901467,
        "incomeTaxExpense": 29749000000,
        "netIncome": 93736000000,
        "netIncomeRatio": 0.2397125577,
        "eps": 6.11,
        "epsdiluted": 6.08,
        "weightedAverageShsOut": 15343783000,
        "weightedAverageShsOutDil": 15408095000,
        "link": "https://www.sec.gov/Archives/edgar/data/320193/000032019324000123/0000320193-24-000123-index.htm",
        "finalLink": "https://www.sec.gov/Archives/edgar/data/320193/000032019324000123/aapl-20240928.htm"
    }
]

### VARIABILI IMPORTI PER BALANCE

Variabile | Descrizione | Frequenza reale

totalAssets | Totale attivo | Trimestrale

totalLiabilities | Totale passivo | Trimestrale

cashAndCashEquivalents | Cassa disponibile | Trimestrale

totalDebt | Totale debito (a breve e lungo termine) | Trimestrale

inventory | Scorte | Trimestrale

shareholderEquity | Capitale proprio | Trimestrale

In [ ]:
# Esempio di output per balance

[
    {
        "date": "2024-09-28",
        "symbol": "AAPL",
        "reportedCurrency": "USD",
        "cik": "0000320193",
        "fillingDate": "2024-11-01",
        "acceptedDate": "2024-11-01 06:01:36",
        "calendarYear": "2024",
        "period": "FY",
        "cashAndCashEquivalents": 29943000000,
        "shortTermInvestments": 35228000000,
        "cashAndShortTermInvestments": 65171000000,
        "netReceivables": 66243000000,
        "inventory": 7286000000,
        "otherCurrentAssets": 14287000000,
        "totalCurrentAssets": 152987000000,
        "propertyPlantEquipmentNet": 45680000000,
        "goodwill": 0,
        "intangibleAssets": 0,
        "goodwillAndIntangibleAssets": 0,
        "longTermInvestments": 91479000000,
        "taxAssets": 19499000000,
        "otherNonCurrentAssets": 55335000000,
        "totalNonCurrentAssets": 211993000000,
        "otherAssets": 0,
        "totalAssets": 364980000000,
        "accountPayables": 68960000000,
        "shortTermDebt": 22511000000,
        "taxPayables": 26601000000,
        "deferredRevenue": 8249000000,
        "otherCurrentLiabilities": 50071000000,
        "totalCurrentLiabilities": 176392000000,
        "longTermDebt": 96548000000,
        "deferredRevenueNonCurrent": 0,
        "deferredTaxLiabilitiesNonCurrent": 0,
        "otherNonCurrentLiabilities": 35090000000,
        "totalNonCurrentLiabilities": 131638000000,
        "otherLiabilities": 0,
        "capitalLeaseObligations": 12430000000,
        "totalLiabilities": 308030000000,
        "preferredStock": 0,
        "commonStock": 83276000000,
        "retainedEarnings": -19154000000,
        "accumulatedOtherComprehensiveIncomeLoss": -7172000000,
        "othertotalStockholdersEquity": 0,
        "totalStockholdersEquity": 56950000000,
        "totalEquity": 56950000000,
        "totalLiabilitiesAndStockholdersEquity": 364980000000,
        "minorityInterest": 0,
        "totalLiabilitiesAndTotalEquity": 364980000000,
        "totalInvestments": 126707000000,
        "totalDebt": 106629000000,
        "netDebt": 76686000000,
        "link": "https://www.sec.gov/Archives/edgar/data/320193/000032019324000123/0000320193-24-000123-index.htm",
        "finalLink": "https://www.sec.gov/Archives/edgar/data/320193/000032019324000123/aapl-20240928.htm"
    }
]

### VARIABILI IMPORTI PER CASHFLOW

Variabile | Descrizione | Frequenza reale

operatingCashFlow | Flussi di cassa dalle operazioni | Trimestrale

capitalExpenditure | Investimenti in capitale (CAPEX) | Trimestrale

freeCashFlow | Cash flow operativo - CAPEX | Trimestrale

dividendsPaid | Dividendi distribuiti | Trimestrale

In [ ]:
# Esempio di output per cashflow
[
    {
        "date": "2024-09-28",
        "symbol": "AAPL",
        "reportedCurrency": "USD",
        "cik": "0000320193",
        "fillingDate": "2024-11-01",
        "acceptedDate": "2024-11-01 06:01:36",
        "calendarYear": "2024",
        "period": "FY",
        "netIncome": 93736000000,
        "depreciationAndAmortization": 11445000000,
        "deferredIncomeTax": 0,
        "stockBasedCompensation": 11688000000,
        "changeInWorkingCapital": 3651000000,
        "accountsReceivables": -5144000000,
        "inventory": -1046000000,
        "accountsPayables": 6020000000,
        "otherWorkingCapital": 3821000000,
        "otherNonCashItems": -2266000000,
        "netCashProvidedByOperatingActivities": 118254000000,
        "investmentsInPropertyPlantAndEquipment": -9447000000,
        "acquisitionsNet": 0,
        "purchasesOfInvestments": -48656000000,
        "salesMaturitiesOfInvestments": 62346000000,
        "otherInvestingActivites": -1308000000,
        "netCashUsedForInvestingActivites": 2935000000,
        "debtRepayment": -5998000000,
        "commonStockIssued": 0,
        "commonStockRepurchased": -94949000000,
        "dividendsPaid": -15234000000,
        "otherFinancingActivites": -5802000000,
        "netCashUsedProvidedByFinancingActivities": -121983000000,
        "effectOfForexChangesOnCash": 0,
        "netChangeInCash": -794000000,
        "cashAtEndOfPeriod": 29943000000,
        "cashAtBeginningOfPeriod": 30737000000,
        "operatingCashFlow": 118254000000,
        "capitalExpenditure": -9447000000,
        "freeCashFlow": 108807000000,
        "link": "https://www.sec.gov/Archives/edgar/data/320193/000032019324000123/0000320193-24-000123-index.htm",
        "finalLink": "https://www.sec.gov/Archives/edgar/data/320193/000032019324000123/aapl-20240928.htm"
    }
]